In [1]:
import numpy as np
import pandas as pd
import nltk
import sys

In [17]:
corpus_v1 = pd.read_excel("corpus_v1.0.xlsx")

In [18]:
corpus_v1

,Interviewer,Interviewee,Task,Sentence,Data Related,Table,Note
0,Xiong Kai,WJC,1,What is the popularity of Arkansas on July 1st...,"popularity, Arkansas, July 1st 2002",co-est2002-01-05,NaN
1,Xiong Kai,WJC,1,Where is Harvard University?,Harvard University,top-50-us-universities,where 表示 location
2,Xiong Kai,WJC,1,How many faculties are there in Harvard Univer...,"faculties, Harvard Universtiy",top-50-us-universities,NaN
3,Xiong Kai,WJC,1,What is the type of Harvard University?,"Harvard University, type",top-50-us-universities,NaN
4,Xiong Kai,WJC,2,which county has more than 30000 popularity?,"county, 30000 popularity",co-est2002-01-05,NaN
...,...,...,...,...,...,...,...
890,Ge Xiaodong,Tang Junxiu,8,Show me the outlier of trip duration between t...,"trip duration, stations",bike,NaN
891,Ge Xiaodong,Tang Junxiu,9,Cluster the trip duration according to start s...,"trip duration, start stations",bike,NaN
892,Ge Xiaodong,Tang Junxiu,9,Is there any clusters of trip durations?,trip durations,bike,NaN
893,Ge Xiaodong,Tang Junxiu,10,Is there any trend of the using frequency of b...,"using frequency, bikes",bike,NaN


In [19]:
len(corpus_v1[(corpus_v1.Task == '1') | (corpus_v1.Task == 1) | corpus_v1.Task.str.contains('1,', case=False, na=False)])

93

In [20]:
corpus_v1.loc[491]

Interviewer                                           Ge Xiaodong
Interviewee                                           Deng Dazhen
Task                                                            1
Sentence        Retrive the neighborhood name with the boy bir...
Data Related        neighborhood name, boy birth number, 56, 2017
Table                                                      births
Note                                                          NaN
Name: 491, dtype: object

In [21]:
tasks_sentences = []
tasks_len = []
for i in range(1,11):
    # tasks_sentences.append([i,str(i),str(i)+','])
    tasks_sentences.append(dict(corpus_v1[(corpus_v1.Task == i) | (corpus_v1.Task == str(i)) | corpus_v1.Task.str.contains(str(i) + ',', case=False, na=False)].Sentence))
    # print(len(tasks_sentences[i-1]))
    tasks_len.append(len(tasks_sentences[i-1]))
    
sum(tasks_len)

895

In [22]:
len(corpus_v1), len(corpus_v1[corpus_v1.Interviewer == 'Ge Xiaodong'])

(895, 405)

In [23]:
tasks_sentences[0]

{0: 'What is the popularity of Arkansas on July 1st 2002?',
 1: 'Where is Harvard University?',
 2: 'How many faculties are there in Harvard Universtiy?',
 3: 'What is the type of Harvard University?',
 42: 'What is the happiness rank of Iceland?',
 43: 'How many dimensions to describe happiness score?',
 44: 'How many students in the grade records?',
 45: "Show me Kibo Mcgee's mid-term score.",
 83: 'What is the revenue of "Titanic"?',
 84: 'How many goals does Liverpool score in the game?',
 85: 'Find the score of course A for student B.',
 86: 'How long would it take from location A to location B?',
 128: 'What region does Iceland belong to?',
 129: 'Which country is the 15th most happy one?',
 130: 'How would people in Iceland rate theirs happiness?',
 131: 'Which country is the next target, if Iceland want to enhance the happiness score?',
 155: 'Which publisher published The Queen of Nothing?',
 156: 'Find me books written by Brian Tracy.',
 157: 'What language is The Art of Supe

In [ ]:
class NGram(object):

    def __init__(self):
        # n is the order of n-gram language model
        self.unigram = {}
        self.bigram = {}
        self.trigram = {}
        self.sentence_len = []

    # scan a sentence, extract the ngram and update their
    # frequence.
    #
    # @param    sentence    list{str}
    # @return   none
    def scan(self, sentences):
        fip = ""
        # file your code here
        for sentence in sentences.values():
            self.sentence_len.append(len(sentence.split()))  # sentence.strip().split(' ')
            self.ngram(sentence.split())
        # unigram
        try:
            fip = open("uni", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.uni")
        tmp = zip(self.unigram.values(),self.unigram.keys())
        for i in sorted(tmp,reverse=True):
            fip.write("%s %d\n" % (i[1], i[0]))
            
        # bigram
        try:
            fip = open("bi", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.bi")
        tmp = zip(self.bigram.values(),self.bigram.keys())
        for i in sorted(tmp,reverse=True):
            fip.write("%s %d\n" % (i[1], i[0]))
            
        # bigram
        try:
            fip = open("tri", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.tri")
        tmp = zip(self.trigram.values(),self.trigram.keys())
        for i in sorted(tmp,reverse=True):
            fip.write("%s %d\n" % (i[1], i[0]))
        
        try:
            fip = open("len", "w", encoding='utf-8')
        except:
            print(sys.stderr, "failed to open data.len")
        fip.write("Sentences Num: %d\n" % (len(self.sentence_len)))
        for i in self.sentence_len:
            fip.write("%d, " % (i))


    # caluclate the ngram of the words
    #
    # @param    words       list{str}
    # @return   none
    def ngram(self, words):
        wlen = len(words)
                  
        for i in range(wlen):
            word = words[i]
                  
            if word not in self.unigram:
                self.unigram[word] = 1
            else:
                self.unigram[word] = self.unigram[word] + 1
            
            if i < (wlen - 1):
                bi_words = ' '.join([word, words[i+1]])
                if bi_words not in self.bigram:
                    self.bigram[bi_words] = 1
                else:
                    self.bigram[bi_words] = self.bigram[bi_words] + 1
                
                if i < (wlen - 2):
                    tri_words = ' '.join([word, words[i+1], words[i+2]])
                    if tri_words not in self.trigram:
                        self.trigram[tri_words] = 1
                    else:
                        self.trigram[tri_words] = self.trigram[tri_words] + 1
                        